In [13]:
%reload_ext autoreload
%autoreload 2

from src.file_utils import download_file, decompress_gz,make_dir
import glob
from warcio.archiveiterator import ArchiveIterator
from bs4 import BeautifulSoup
from tqdm import tqdm
from src.parse_warc import extract_japanese_from_warc

import json
import gzip

In [14]:
path="data/jap_dump/crawl-data_CC-MAIN-2023-50_segments_1700679099281.67_warc_CC-MAIN-20231128083443-20231128113443-00000.warc_jap.gz.gz"
# gzipで圧縮されたJSONファイルを読み込み
with gzip.open(path, 'rt', encoding='utf-8') as f:
    data_loaded = json.load(f)


In [59]:
"""
from bunkai import Bunkai
bunkai = Bunkai()
def parse_lines_bunkai(lines):
    lines = lines.strip().replace("\n", "\u2581")
    return bunkai("".join(lines))

"""

'\nfrom bunkai import Bunkai\nbunkai = Bunkai()\ndef parse_lines_bunkai(lines):\n    lines = lines.strip().replace("\n", "▁")\n    return bunkai("".join(lines))\n\n'

In [100]:
from src.cleaner.auto_filter import do_filter
from src.cleaner.text_normalizer import normalize

lines=data_loaded[20]["text"]
lines=[i[0] for i in lines]
text="".join(lines)


# 文末記号のリスト（全角と半角）
sentence_endings = ['。', '！', '？', '.', '!', '?',"．"]
touten_symbols=["､",",","，","、"]

def is_sentence_ending(text):
    """指定された文字が文末記号かどうかを判断する"""
    return text[-1] in sentence_endings

# 結果を格納するリスト

grouped_lines=[]

temp_text=""
old_line=""
for line in lines:
    line=normalize(line) #文字の正規化

    #重複行は削除
    if line==old_line:
        old_line=line
        continue

    old_line=line

    if old_line[-1] in touten_symbols:
        temp_text+=line
        #print(temp_text)
        continue

    # ｡ などで終わらないlineの場合は､改行する
    if not is_sentence_ending(line):
        if temp_text!="":
            grouped_lines.append(temp_text)
            #print(temp_text)
        temp_text=""
        grouped_lines.append(line)
        #print(line)

    else:
        temp_text+=line
    #if not is_sentence_ending(temp_text):
    #    grouped_lines.append(temp_text)
    #    temp_text=""

text="\n".join(grouped_lines)
text=do_filter(text)
print(text)


#TODO: まず、  をしてください(無料)。 のようになってしまう

お問い合わせ
同一のユーザが1日の間に何ページ読み込もうと、ユニークカウント(ビジット数)は1と数える方法。実際の閲覧ユーザ数に近い人数を数える事ができます。
お天気ガイド
過去のリンク集
です。ご覧になりたい地域をクリックしてください。
ご自分の、または、お勧めのホームページを登録したり
掲示板に記事を投稿したりするには、まず、
をしてください(無料)。
詳しい登録方法は
。※このページは古い過去の情報ですので修正・削除などメンテナンスされていません。
リンク切れ等ご了承下さい。新しい情報は
www-staff@(this-server), 2003/07/26.※このページは古い過去の情報ですので修正・削除などメンテナンスされていません。
※このページは古い過去の情報ですので修正・削除などメンテナンスされていません。


In [98]:
line="まず、"
line[-1] in touten_symbols

True

In [90]:
line[-1]

'、'